In [447]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /Users/wjones/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [448]:
# Data Preprocessing
path = "RottenTomatoes/DataSet/train.tsv"
df = pd.read_csv(path, sep="\t") 

# remove punctuation
df['Phrase'] = df['Phrase'].str.replace(r'[^\w\s]+', '')
# remove numbers
df['Phrase'] = df['Phrase'].str.replace(r'\d+', '')
# make it all lower case
df['Phrase'] = df['Phrase'].str.lower()
# remove non-asci characters
df.Phrase.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
#df['Phrase'] = df['Phrase'].str.split()

df['Tokenized_text'] = df['Phrase'].apply(word_tokenize) 

df['Sentiment']=df['Sentiment'].astype(int) #convert the star_rating column to int
df['NNLabels'] = df['Sentiment'].div(4)

df= df[df['Sentiment']!=2]

#df['label']=np.where(df['Sentiment']>=4,1,0) #1-Positve,0-Negative
# convert to NumPy Array
train = df['Phrase'].to_numpy()


/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_16638/3302568041.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Phrase'] = df['Phrase'].str.replace(r'[^\w\s]+', '')
/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_16638/3302568041.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Phrase'] = df['Phrase'].str.replace(r'\d+', '')


In [449]:
df

,PhraseId,SentenceId,Phrase,Sentiment,Tokenized_text,NNLabels
0,1,1,a series of escapades demonstrating the adage ...,1,"[a, series, of, escapades, demonstrating, the,...",0.25
21,22,1,good for the goose,3,"[good, for, the, goose]",0.75
22,23,1,good,3,[good],0.75
33,34,1,the gander some of which occasionally amuses ...,1,"[the, gander, some, of, which, occasionally, a...",0.25
46,47,1,amuses,3,[amuses],0.75
...,...,...,...,...,...,...
156047,156048,8544,quietly suggesting the sadness and obsession b...,1,"[quietly, suggesting, the, sadness, and, obses...",0.25
156051,156052,8544,sadness and obsession,1,"[sadness, and, obsession]",0.25
156052,156053,8544,sadness and,1,"[sadness, and]",0.25
156056,156057,8544,forced avuncular chortles,1,"[forced, avuncular, chortles]",0.25


In [450]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(df['Phrase'], df['Sentiment'], test_size=0.2, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train:  (61182,) (61182,) Test:  ((15296,), (15296,))


In [451]:
X_train,X_valid,Y_train, Y_valid = train_test_split(X_train,Y_train, test_size=0.5, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test: ",(X_valid.shape,Y_valid.shape))

Train:  (30591,) (30591,) Test:  ((30591,), (30591,))


In [452]:
from nltk.stem.porter import *

In [453]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [454]:
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')


In [455]:
count_wm = countvectorizer.fit_transform(X_train)
tfidf_wm = tfidfvectorizer.fit_transform(X_train)

In [456]:
count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()

/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [457]:
df_countvect = pd.DataFrame(data = count_wm.toarray(),columns = count_tokens)
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)

tf_x_train = tfidfvectorizer.fit_transform(X_train)
tf_x_valid = tfidfvectorizer.transform(X_valid)



In [481]:
tf_x_train

<30591x14320 sparse matrix of type '<class 'numpy.float64'>'
	with 140117 stored elements in Compressed Sparse Row format>

In [485]:
print(tf_x_valid.toarray)

<bound method _cs_matrix.toarray of <30591x14320 sparse matrix of type '<class 'numpy.float64'>'
	with 136393 stored elements in Compressed Sparse Row format>>


In [459]:

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [460]:
kmeans = KMeans(init="k-means++", n_clusters=5, random_state=0)


In [487]:
kmeans.fit(tf_x_train.toarray())
preds = kmeans.predict(tf_x_valid.toarray())

In [462]:
import matplotlib.pyplot as plt


In [463]:
#plt.scatter(tf_x_valid[:,0], tf_x_valid[:,1], c=preds)

from 

In [527]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
arr = []
preds = np.array(preds)
for i in preds:
    arr.append(i)
#print(arr)
#print(Y_valid)
#print(Y_valid*4)
report=classification_report(Y_valid.array*4, arr,output_dict=True)

/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [528]:
report

{'0.0': {'precision': 0.15695952615992104,
  'recall': 0.05584826132771338,
  'f1-score': 0.08238341968911918,
  'support': 2847},
 '1.0': {'precision': 0.2558011049723757,
  'recall': 0.04242646385045359,
  'f1-score': 0.07278157667216852,
  'support': 10913},
 '2.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0},
 '3.0': {'precision': 0.6541850220264317,
  'recall': 0.02267175572519084,
  'f1-score': 0.04382470119521912,
  'support': 13100},
 '4.0': {'precision': 0.1487481590574374,
  'recall': 0.05414098097024926,
  'f1-score': 0.07938691294949891,
  'support': 3731},
 'accuracy': 0.036644764800104604,
 'macro avg': {'precision': 0.24313876244323315,
  'recall': 0.03501749237472142,
  'f1-score': 0.055675322101201144,
  'support': 30591},
 'weighted avg': {'precision': 0.40414580757511637,
  'recall': 0.036644764800104604,
  'f1-score': 0.06208058252264549,
  'support': 30591}}

In [489]:
report

{'0': {'precision': 0.08965414319292617,
  'recall': 0.8440463645943098,
  'f1-score': 0.16209106239460372,
  'support': 2847},
 '1': {'precision': 0.352112676056338,
  'recall': 0.045816915605241454,
  'f1-score': 0.08108327252087894,
  'support': 10913},
 '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0},
 '3': {'precision': 0.5243619489559165,
  'recall': 0.017251908396946566,
  'f1-score': 0.033404774222156534,
  'support': 13100},
 '4': {'precision': 0.46048109965635736,
  'recall': 0.035915304207987137,
  'f1-score': 0.06663351566384883,
  'support': 3731},
 'accuracy': 0.10666535909254356,
 'macro avg': {'precision': 0.2853219735723076,
  'recall': 0.188606098560897,
  'f1-score': 0.0686425249602976,
  'support': 30591},
 'weighted avg': {'precision': 0.41466599632615647,
  'recall': 0.10666535909254356,
  'f1-score': 0.06644265296688108,
  'support': 30591}}

In [490]:
from sklearn.metrics.pairwise import euclidean_distances

In [491]:
centroids = kmeans.cluster_centers_

In [492]:
print(centroids)

[[ 4.06575815e-20  5.42101086e-20 -1.08420217e-19 ...  2.37169225e-20
  -1.01643954e-20  5.08219768e-20]
 [-8.13151629e-20 -3.04931861e-20 -1.30104261e-18 ... -6.43745040e-20
  -4.40457133e-20 -9.48676901e-20]
 [ 3.82266391e-05  2.62399082e-05  3.35495622e-04 ...  1.91894622e-05
   1.05589277e-05  3.23583135e-05]
 [-2.71050543e-20  0.00000000e+00  3.25260652e-19 ...  1.69406589e-20
   0.00000000e+00  6.77626358e-21]
 [-3.38813179e-20  5.75982404e-20  8.82497467e-04 ... -3.04931861e-20
  -3.72694497e-20 -1.69406589e-20]]


In [469]:
new_x_train_features = kmeans.fit_transform(tf_x_train)
new_x_valid_features = kmeans.fit_transform(tf_x_valid)

In [470]:
Y_train/=4   
Y_valid/=4

In [471]:
Y_train.array

<PandasArray>
[0.75, 0.25, 0.75, 0.25, 0.75, 0.25,  1.0, 0.25, 0.25,  1.0,
 ...
 0.75, 0.75,  1.0,  0.0, 0.25, 0.75, 0.25, 0.75, 0.25, 0.75]
Length: 30591, dtype: float64

In [472]:
#import torch

import torch.nn as nn

# Define the number of input features
K = 5

# Define the first linear layer
linear_layer1 = nn.Linear(K, 32)

# Define the second linear layer
linear_layer2 = nn.Linear(32, 1)

# Print the two linear layers
print(linear_layer1)
print(linear_layer2)

model = torch.nn.Sequential(
    linear_layer1,
    torch.nn.Sigmoid(),
    linear_layer2,
    torch.nn.Sigmoid()
)

Y_train_array = np.asarray(Y_train).astype(float)
X_train_array = np.asarray(new_x_train_features).astype(float)
X_valid_array= np.asarray(new_x_valid_features)

Y_tensor = torch.tensor(Y_train_array).float()
X_tensor = torch.tensor(X_train_array).float()
X_valid_tensor = torch.tensor(X_valid_array).float()

new_x_valid_features.shape

# make a dataset object
data = torch.utils.data.TensorDataset(X_tensor, Y_tensor)

# then use it to make a dataloader object
loader = torch.utils.data.DataLoader(data, batch_size=32, shuffle=True)

# make a scoring function
scorer = torch.nn.MSELoss()

# make optimizer

opt = torch.optim.Adam(params = model.parameters(), lr=.001)

num_epochs = 100

for i in range(num_epochs):
    for batch in loader:
        # unpack data
        batchX = batch[0]
        batchY = batch[1]

        # zero out last run 
        opt.zero_grad

        # get the predicitions
        preds = model(batchX)

        # get the error
        err = scorer(preds, batchY)

        # send the gradient information to the parameters
        err.backward()

        # take the step
        opt.step()

In [473]:
#preds *= 4
##preds = preds.round(decimals=0, out=None)
#Y_train *=4

report=classification_report(Y_train, preds,output_dict=True)
#from sklearn.metrics import mean_absolute_error
#(Y_train, preds)

In [474]:
report

{'0': {'precision': 0.08965414319292617,
  'recall': 0.8440463645943098,
  'f1-score': 0.16209106239460372,
  'support': 2847},
 '1': {'precision': 0.352112676056338,
  'recall': 0.045816915605241454,
  'f1-score': 0.08108327252087894,
  'support': 10913},
 '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0},
 '3': {'precision': 0.5243619489559165,
  'recall': 0.017251908396946566,
  'f1-score': 0.033404774222156534,
  'support': 13100},
 '4': {'precision': 0.46048109965635736,
  'recall': 0.035915304207987137,
  'f1-score': 0.06663351566384883,
  'support': 3731},
 'accuracy': 0.10666535909254356,
 'macro avg': {'precision': 0.2853219735723076,
  'recall': 0.188606098560897,
  'f1-score': 0.0686425249602976,
  'support': 30591},
 'weighted avg': {'precision': 0.41466599632615647,
  'recall': 0.10666535909254356,
  'f1-score': 0.06644265296688108,
  'support': 30591}}

In [475]:
import torch
import torch.nn as nn
# Define the number of input features
K = 5

# Define the first linear layer
linear_layer1 = nn.Linear(K, 64)

# Define the second linear layer
linear_layer2 = nn.Linear(64, 5)

# Print the two linear layers
print(linear_layer1)
print(linear_layer2)

model = torch.nn.Sequential(
    linear_layer1,
    torch.nn.Sigmoid(),
    linear_layer2,
    torch.nn.Sigmoid()
)

Y_train_array = np.asarray(Y_train).astype(float)
X_train_array = np.asarray(new_x_train_features.toarray()).astype(float)
X_valid_array= np.asarray(new_x_valid_features.toarray()).astype(float)
Y_valid_array = np.asarray(Y_valid).astype(float)

Y_tensor = torch.tensor(Y_train_array).long()
X_tensor = torch.tensor(X_train_array).float()
X_valid_tensor = torch.tensor(X_valid_array).float()
Y_valid_tensor = torch.tensor(Y_valid_array).long()

#print(X_tensor.shape)
# make a dataset object
data = torch.utils.data.TensorDataset(X_tensor, Y_tensor)

# then use it to make a dataloader object
loader = torch.utils.data.DataLoader(data, batch_size=32, shuffle=True)

# make a scoring function
scorer = torch.nn.NLLLoss()

opt = torch.optim.Adam( model.parameters(), 0.000001   )

num_epochs = 10
for j in range(num_epochs):
    for batch in loader:
        # unpack data
        batchX = batch[0]

        batchY = batch[1]

        #print(batchY.shape)
        # zero out last run 
        opt.zero_grad

        # get the predicitions
        preds = model(batchX)
     
        # get the error
        #print(preds.shape)
        err = scorer(preds, batchY)
        
        # send the gradient information to the parameters
        err.backward()

        # take the step
        opt.step()

preds = model(X_tensor).detach().numpy()
print(preds)

#plt.scatter(trainX[:,0], trainX[:, 1], c=pred_labels)

Linear(in_features=5, out_features=64, bias=True)
Linear(in_features=64, out_features=5, bias=True)
[[0.5174502  0.59626025 0.5423946  0.4381323  0.522819  ]
 [0.51744306 0.5962762  0.54238534 0.43812832 0.5228115 ]
 [0.51744473 0.5962698  0.5423898  0.43813005 0.52281237]
 ...
 [0.51743156 0.5962236  0.54242563 0.43812603 0.5228912 ]
 [0.51744264 0.59627753 0.5423826  0.43812978 0.5228086 ]
 [0.5174229  0.5962624  0.54240173 0.43808675 0.52284086]]


In [ ]:
test_output_labels = np.argmax(preds,1)
print(test_output_labels)

In [477]:

#pred_labels = pred_labels.round(decimals=0, out=None)


In [478]:
for i in pred_labels:
    print(i)

0.59626025
0.5962762
0.5962698
0.5962321
0.5962629
0.5962746
0.5958501
0.59625983
0.5962463
0.5962595
0.59612316
0.59625494
0.59623873
0.5962311
0.59626
0.5962431
0.5962462
0.5961783
0.5962689
0.5962685
0.5951061
0.5962272
0.5962133
0.596226
0.5960835
0.59626144
0.5962658
0.5962742
0.59627366
0.59626746
0.59548515
0.595781
0.5960745
0.5962239
0.5962718
0.59625185
0.5961928
0.59626883
0.5962484
0.5962601
0.5962036
0.5962735
0.59626055
0.5962038
0.59627116
0.59659463
0.59613985
0.59658307
0.5962715
0.59626794
0.5962407
0.5962438
0.59627104
0.59623164
0.596267
0.5961534
0.596262
0.5962741
0.59626114
0.59627205
0.59619975
0.59625685
0.5962752
0.5962252
0.5962371
0.59625775
0.59548515
0.5962461
0.5962412
0.5955804
0.5962799
0.59627444
0.596279
0.5960139
0.5956171
0.5962417
0.5962614
0.5962739
0.5962548
0.5962231
0.59626263
0.5962409
0.59625185
0.5962754
0.5962731
0.5962709
0.5962539
0.59616613
0.5962676
0.5962701
0.59622914
0.59624517
0.5962167
0.5962359
0.5962403
0.59624714
0.5962576
0.596

In [480]:
y_arr = np.array(Y_train)*4
report=classification_report(y_arr, preds,output_dict=True)


ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

SyntaxError: 'function call' is an illegal expression for augmented assignment (936699697.py, line 1)

In [ ]:
print(pred_labels)

[0.6913326  0.6913357  0.6913394  ... 0.69131166 0.6913356  0.69135463]
